In [ ]:
mode = 'all' #pick one of enby, all

In [ ]:
import bs4, os, pickle, pytumblr, requests, shutil

def write_tumblr_img(url, path):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(path, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)    

# Connect to tumblr API

In [ ]:
client = pytumblr.TumblrRestClient() #get key from https://api.tumblr.com/console

assert client.info() # Confirm connection

# Get "selfie" tagged posts

In [ ]:
def get_posts(n_pages, tag):
    before_start = None #increment by 20 for each page
    all_posts = []
    selfie_posts = []
    for page in range(n_pages):
        posts = client.tagged(tag, limit=20, before=before_start)
        all_posts+=posts
        before_start = posts[-1]['timestamp']
        for post in posts:
            if 'selfie' in post['tags']:
                selfie_posts.append(post) #this will be equivalent to all_posts for "selfie" tagged images, but only a subset for "enby" tagged images
    return all_posts, selfie_posts
if mode=='enby':
    all_posts, selfie_posts = get_posts(600, 'enby')
if mode=='all':
    all_posts, selfie_posts = get_posts(30, 'selfie')

# Extract images from selfie posts

In [ ]:
results = {}
for post in selfie_posts:
    try:
        results[post['id']] = (post, 
                               ((post['photos'][0]['alt_sizes'][2]['url']))) #extract photo using Tumblr's API
    except: 
        try:
            results[post['id']] = (post, 
                                   (bs4.BeautifulSoup(post['body']).find('img')['src'])) #parse and extract text page
        except IndexError:
            print("Can't locate photo", selfie_posts.index(post)) #can't find photo

# Save image information

In [5]:
if mode =='all': base_dir = 'out_all'
if mode =='enby': base_dir = 'out_enby'
if 'out_all' not in os.listdir('./'): os.mkdir('out_all')
if 'out_enby' not in os.listdir('./'): os.mkdir('out_enby')
for result in results:
    if results[result][1].split('/')[-1] not in os.listdir(base_dir):
        write_tumblr_img(results[result][1],
                        '%s/%s' % (base_dir, results[result][1].split('/')[-1]))

# Save post information

In [ ]:
if mode == 'enby':
    pickle.dump(selfie_posts, open('selfie_posts_enby.pkl','wb'))
if mode == 'all':
    pickle.dump(selfie_posts, open('selfie_posts_all.pkl','wb'))
